In [1]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

2022-03-24 16:38:15.065152: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 16:38:15.096108: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-03-24 16:38:15.096123: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-03-24 16:38:15.096511: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [2]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]


In [3]:
#k_folds = utils.create_k_folds_n_users(5, 40, '/home/wander/OtherProjects/har_flower/full_data')

In [4]:
paths = ['/home/wander/OtherProjects/har_flower/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv', 
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv']
bas = []

for path in paths:

    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            'neurons_1' : 32, 
            'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    test_results, ba = har.run()
    #model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)
print(max(bas))


Epoch 1/40
254/254 [==============================] - 1s 2ms/step - loss: 692910.6875 - avg_multilabel_BA_2: 0.7538 - val_loss: 0.4069 - val_avg_multilabel_BA_2: 0.7473
Epoch 2/40
254/254 [==============================] - 0s 967us/step - loss: 0.2808 - avg_multilabel_BA_2: 0.7536 - val_loss: 0.3945 - val_avg_multilabel_BA_2: 0.7471
Epoch 3/40
254/254 [==============================] - 0s 933us/step - loss: 0.2816 - avg_multilabel_BA_2: 0.7520 - val_loss: 0.3885 - val_avg_multilabel_BA_2: 0.7472
Epoch 4/40
254/254 [==============================] - 0s 896us/step - loss: 0.2827 - avg_multilabel_BA_2: 0.7505 - val_loss: 0.3989 - val_avg_multilabel_BA_2: 0.7472
Epoch 5/40
254/254 [==============================] - 0s 907us/step - loss: 0.2808 - avg_multilabel_BA_2: 0.7500 - val_loss: 0.3325 - val_avg_multilabel_BA_2: 0.7473
Epoch 6/40
254/254 [==============================] - 0s 931us/step - loss: 0.2831 - avg_multilabel_BA_2: 0.7494 - val_loss: 0.3637 - val_avg_multilabel_BA_2: 0.7473
E

Epoch 9/40
199/199 [==============================] - 0s 887us/step - loss: 0.2731 - avg_multilabel_BA_2: 0.7228 - val_loss: 0.2828 - val_avg_multilabel_BA_2: 0.7215
Epoch 10/40
199/199 [==============================] - 0s 889us/step - loss: 0.2740 - avg_multilabel_BA_2: 0.7203 - val_loss: 0.2826 - val_avg_multilabel_BA_2: 0.7190
Epoch 11/40
199/199 [==============================] - 0s 888us/step - loss: 0.2731 - avg_multilabel_BA_2: 0.7177 - val_loss: 0.2743 - val_avg_multilabel_BA_2: 0.7165
Epoch 12/40
199/199 [==============================] - 0s 909us/step - loss: 0.2734 - avg_multilabel_BA_2: 0.7153 - val_loss: 0.2750 - val_avg_multilabel_BA_2: 0.7141
Epoch 13/40
199/199 [==============================] - 0s 918us/step - loss: 0.2727 - avg_multilabel_BA_2: 0.7129 - val_loss: 0.2894 - val_avg_multilabel_BA_2: 0.7117
Epoch 14/40
199/199 [==============================] - 0s 933us/step - loss: 0.2743 - avg_multilabel_BA_2: 0.7105 - val_loss: 0.2653 - val_avg_multilabel_BA_2: 0.7094

Epoch 17/40
199/199 [==============================] - 0s 908us/step - loss: 0.2736 - avg_multilabel_BA_2: 0.6419 - val_loss: 0.2769 - val_avg_multilabel_BA_2: 0.6414
Epoch 18/40
199/199 [==============================] - 0s 898us/step - loss: 0.2732 - avg_multilabel_BA_2: 0.6409 - val_loss: 0.2760 - val_avg_multilabel_BA_2: 0.6404
Epoch 19/40
199/199 [==============================] - 0s 887us/step - loss: 0.2724 - avg_multilabel_BA_2: 0.6398 - val_loss: 0.2645 - val_avg_multilabel_BA_2: 0.6393
Epoch 20/40
199/199 [==============================] - 0s 894us/step - loss: 0.2745 - avg_multilabel_BA_2: 0.6388 - val_loss: 0.2815 - val_avg_multilabel_BA_2: 0.6383
Epoch 21/40
199/199 [==============================] - 0s 955us/step - loss: 0.2732 - avg_multilabel_BA_2: 0.6378 - val_loss: 0.2711 - val_avg_multilabel_BA_2: 0.6373
Epoch 22/40
199/199 [==============================] - 0s 986us/step - loss: 0.2729 - avg_multilabel_BA_2: 0.6368 - val_loss: 0.2811 - val_avg_multilabel_BA_2: 0.636

Epoch 25/40
199/199 [==============================] - 0s 936us/step - loss: 0.2728 - avg_multilabel_BA_2: 0.6041 - val_loss: 0.2875 - val_avg_multilabel_BA_2: 0.6038
Epoch 26/40
199/199 [==============================] - 0s 925us/step - loss: 0.2734 - avg_multilabel_BA_2: 0.6035 - val_loss: 0.2614 - val_avg_multilabel_BA_2: 0.6032
Epoch 27/40
199/199 [==============================] - 0s 924us/step - loss: 0.2731 - avg_multilabel_BA_2: 0.6029 - val_loss: 0.2834 - val_avg_multilabel_BA_2: 0.6027
Epoch 28/40
199/199 [==============================] - 0s 936us/step - loss: 0.2730 - avg_multilabel_BA_2: 0.6024 - val_loss: 0.2748 - val_avg_multilabel_BA_2: 0.6021
Epoch 29/40
199/199 [==============================] - 0s 942us/step - loss: 0.2736 - avg_multilabel_BA_2: 0.6018 - val_loss: 0.2723 - val_avg_multilabel_BA_2: 0.6016
Epoch 30/40
199/199 [==============================] - 0s 940us/step - loss: 0.2733 - avg_multilabel_BA_2: 0.6013 - val_loss: 0.2765 - val_avg_multilabel_BA_2: 0.601

In [5]:
'''def nan_mse(y_actual, y_predicted):
    stack = tf.stack((tf.math.is_nan(y_actual), 
                      tf.math.is_nan(y_predicted)),
                     axis=1)
    is_nans = tf.keras.backend.any(stack, axis=1)
    per_instance = tf.where(is_nans,
                            tf.zeros_like(y_actual),
                            tf.square(tf.subtract(y_predicted, y_actual)))
    print(stack)
    print(per_instance)
    return tf.reduce_mean(per_instance, axis=0)

print(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'''

'def nan_mse(y_actual, y_predicted):\n    stack = tf.stack((tf.math.is_nan(y_actual), \n                      tf.math.is_nan(y_predicted)),\n                     axis=1)\n    is_nans = tf.keras.backend.any(stack, axis=1)\n    per_instance = tf.where(is_nans,\n                            tf.zeros_like(y_actual),\n                            tf.square(tf.subtract(y_predicted, y_actual)))\n    print(stack)\n    print(per_instance)\n    return tf.reduce_mean(per_instance, axis=0)\n\nprint(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'

In [6]:
paths = ['/home/wander/OtherProjects/har_flower/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv', 
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/wander/OtherProjects/har_flower/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv']
bas = []

for path in paths:

    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            #'neurons_1' : 32, 
            #'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    #test_results, ba = har.run()
    model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)

Trial 100 Complete [00h 00m 02s]
val_avg_multilabel_BA_2: 0.7213355898857117

Best val_avg_multilabel_BA_2 So Far: 0.7213821411132812
Total elapsed time: 00h 09m 37s
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
51/51 [==============================] - 1s 8ms/step - loss: 3626848.0000 - avg_multilabel_BA_2: 0.7212 - val_loss: 0.6675 - val_avg_multilabel_BA_2: 0.7212
Epoch 2/100
51/51 [==============================] - 0s 1ms/step - loss: 0.6346 - avg_multilabel_BA_2: 0.7212 - val_loss: 0.6172 - val_avg_multilabel_BA_2: 0.7212
Epoch 3/100
51/51 [==============================] - 0s 1ms/step - loss: 0.5739 - avg_multilabel_BA_2: 0.7212 - val_loss: 0.5686 - val_avg_multilabel_BA_2: 0.7212
Epoch 4/100
51/51 [==============================] - 0s 1ms/step - loss: 0.5179 - avg_multilabel_BA_2: 0.7212 - val_loss: 0.5276 - val_avg_multilabel_BA_2: 0.7212
Epoch 5/100
51/51 [==============================] - 0s 2ms/step - loss: 0.4707 - avg_multilabel_BA_2: 0.7212 - val_loss: 0.4953 - val_avg

Epoch 50/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2832 - avg_multilabel_BA_2: 0.7215 - val_loss: 0.3764 - val_avg_multilabel_BA_2: 0.7215
Epoch 51/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2830 - avg_multilabel_BA_2: 0.7215 - val_loss: 0.3759 - val_avg_multilabel_BA_2: 0.7215
Epoch 52/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2830 - avg_multilabel_BA_2: 0.7215 - val_loss: 0.3765 - val_avg_multilabel_BA_2: 0.7215
Epoch 53/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2828 - avg_multilabel_BA_2: 0.7215 - val_loss: 0.3758 - val_avg_multilabel_BA_2: 0.7215
Epoch 54/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2827 - avg_multilabel_BA_2: 0.7215 - val_loss: 0.3757 - val_avg_multilabel_BA_2: 0.7215
Epoch 55/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2826 - avg_multilabel_BA_2: 0.7215 - val_loss: 0.3753 - val_avg_multilabel_BA_2: 0.7215
Epoch 56/100
51/

Epoch 100/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2803 - avg_multilabel_BA_2: 0.7217 - val_loss: 0.3731 - val_avg_multilabel_BA_2: 0.7217
Best epoch: 100
Epoch 1/100
51/51 [==============================] - 1s 7ms/step - loss: 3422608.7500 - avg_multilabel_BA_2: 0.7218 - val_loss: 0.6643 - val_avg_multilabel_BA_2: 0.7217
Epoch 2/100
51/51 [==============================] - 0s 1ms/step - loss: 0.6310 - avg_multilabel_BA_2: 0.7218 - val_loss: 0.6145 - val_avg_multilabel_BA_2: 0.7218
Epoch 3/100
51/51 [==============================] - 0s 1ms/step - loss: 0.5711 - avg_multilabel_BA_2: 0.7218 - val_loss: 0.5668 - val_avg_multilabel_BA_2: 0.7218
Epoch 4/100
51/51 [==============================] - 0s 1ms/step - loss: 0.5158 - avg_multilabel_BA_2: 0.7218 - val_loss: 0.5263 - val_avg_multilabel_BA_2: 0.7218
Epoch 5/100
51/51 [==============================] - 0s 1ms/step - loss: 0.4692 - avg_multilabel_BA_2: 0.7218 - val_loss: 0.4945 - val_avg_multilabel_BA_2: 0.721

Epoch 50/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2832 - avg_multilabel_BA_2: 0.7220 - val_loss: 0.3759 - val_avg_multilabel_BA_2: 0.7220
Epoch 51/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2830 - avg_multilabel_BA_2: 0.7220 - val_loss: 0.3761 - val_avg_multilabel_BA_2: 0.7220
Epoch 52/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2829 - avg_multilabel_BA_2: 0.7220 - val_loss: 0.3760 - val_avg_multilabel_BA_2: 0.7220
Epoch 53/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2828 - avg_multilabel_BA_2: 0.7220 - val_loss: 0.3764 - val_avg_multilabel_BA_2: 0.7220
Epoch 54/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2827 - avg_multilabel_BA_2: 0.7220 - val_loss: 0.3754 - val_avg_multilabel_BA_2: 0.7220
Epoch 55/100
51/51 [==============================] - 0s 1ms/step - loss: 0.2826 - avg_multilabel_BA_2: 0.7220 - val_loss: 0.3754 - val_avg_multilabel_BA_2: 0.7220
Epoch 56/100
51/

Epoch 100/100
25/25 [==============================] - 0s 692us/step - loss: 0.2983 - avg_multilabel_BA_2: 0.7223
Test results - Loss: 0.29829564690589905 - Averaged Balanced Accuracy: 0.722281277179718%
Averaged Balanced Accuracy: 0.722282
INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1_RandSearchCV/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
40/40 [==============================] - 1s 9ms/step - loss: 4738011.5000 - avg_multilabel_BA_2: 0.7223 - val_loss: 0.6562 - val_avg_multilabel_BA_2: 0.7223
Epoch 2/100
40/40 [==============================] - 0s 1ms/step - loss: 0.6154 - avg_multilabel_BA_2: 0.7223 - val_loss: 0.5661 - val_avg_multilabel_BA_2: 0.7222
Epoch 3/100
40/40 [==============================] - 0s 1ms/step - loss: 0.5307 - avg_multilabel_BA_2: 0.7222 - val_loss: 0.4849 - val_avg_multilabel_BA_2: 0.7222
Epoch 4/100
40/40 [===========

...

40/40 [==============================] - 0s 1ms/step - loss: 0.2705 - avg_multilabel_BA_2: 0.7200 - val_loss: 0.2774 - val_avg_multilabel_BA_2: 0.7200
Epoch 98/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2705 - avg_multilabel_BA_2: 0.7200 - val_loss: 0.2773 - val_avg_multilabel_BA_2: 0.7200
Epoch 99/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2705 - avg_multilabel_BA_2: 0.7200 - val_loss: 0.2776 - val_avg_multilabel_BA_2: 0.7200
Epoch 100/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2705 - avg_multilabel_BA_2: 0.7200 - val_loss: 0.2774 - val_avg_multilabel_BA_2: 0.7199
Best epoch: 1
20/20 [==============================] - 0s 725us/step - loss: 0.6147 - avg_multilabel_BA_2: 0.7199
Test results - Loss: 0.6146687865257263 - Averaged Balanced Accuracy: 0.7199203372001648%
Averaged Balanced Accuracy: 0.719913
INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:ten

40/40 [==============================] - 0s 1ms/step - loss: 0.2751 - avg_multilabel_BA_2: 0.7189 - val_loss: 0.2782 - val_avg_multilabel_BA_2: 0.7189
Epoch 44/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2749 - avg_multilabel_BA_2: 0.7189 - val_loss: 0.2779 - val_avg_multilabel_BA_2: 0.7189
Epoch 45/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2748 - avg_multilabel_BA_2: 0.7189 - val_loss: 0.2780 - val_avg_multilabel_BA_2: 0.7189
Epoch 46/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2747 - avg_multilabel_BA_2: 0.7189 - val_loss: 0.2777 - val_avg_multilabel_BA_2: 0.7189
Epoch 47/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2746 - avg_multilabel_BA_2: 0.7188 - val_loss: 0.2775 - val_avg_multilabel_BA_2: 0.7188
Epoch 48/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2744 - avg_multilabel_BA_2: 0.7188 - val_loss: 0.2776 - val_avg_multilabel_BA_2: 0.7188
Epoch 49/100
40/40 [=========

40/40 [==============================] - 0s 1ms/step - loss: 0.2725 - avg_multilabel_BA_2: 0.7178 - val_loss: 0.2750 - val_avg_multilabel_BA_2: 0.7178
Epoch 94/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2725 - avg_multilabel_BA_2: 0.7178 - val_loss: 0.2748 - val_avg_multilabel_BA_2: 0.7178
Epoch 95/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2725 - avg_multilabel_BA_2: 0.7178 - val_loss: 0.2748 - val_avg_multilabel_BA_2: 0.7177
Epoch 96/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2724 - avg_multilabel_BA_2: 0.7177 - val_loss: 0.2748 - val_avg_multilabel_BA_2: 0.7177
Epoch 97/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2724 - avg_multilabel_BA_2: 0.7177 - val_loss: 0.2749 - val_avg_multilabel_BA_2: 0.7177
Epoch 98/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2724 - avg_multilabel_BA_2: 0.7177 - val_loss: 0.2748 - val_avg_multilabel_BA_2: 0.7177
Epoch 99/100
40/40 [=========

40/40 [==============================] - 0s 1ms/step - loss: 0.2781 - avg_multilabel_BA_2: 0.7167 - val_loss: 0.2871 - val_avg_multilabel_BA_2: 0.7167
Epoch 40/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2778 - avg_multilabel_BA_2: 0.7167 - val_loss: 0.2870 - val_avg_multilabel_BA_2: 0.7167
Epoch 41/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2775 - avg_multilabel_BA_2: 0.7167 - val_loss: 0.2865 - val_avg_multilabel_BA_2: 0.7167
Epoch 42/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2771 - avg_multilabel_BA_2: 0.7167 - val_loss: 0.2861 - val_avg_multilabel_BA_2: 0.7167
Epoch 43/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2769 - avg_multilabel_BA_2: 0.7166 - val_loss: 0.2856 - val_avg_multilabel_BA_2: 0.7166
Epoch 44/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2766 - avg_multilabel_BA_2: 0.7166 - val_loss: 0.2853 - val_avg_multilabel_BA_2: 0.7166
Epoch 45/100
40/40 [=========

40/40 [==============================] - 0s 1ms/step - loss: 0.2724 - avg_multilabel_BA_2: 0.7156 - val_loss: 0.2797 - val_avg_multilabel_BA_2: 0.7156
Epoch 90/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2723 - avg_multilabel_BA_2: 0.7156 - val_loss: 0.2795 - val_avg_multilabel_BA_2: 0.7156
Epoch 91/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2723 - avg_multilabel_BA_2: 0.7156 - val_loss: 0.2795 - val_avg_multilabel_BA_2: 0.7156
Epoch 92/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2723 - avg_multilabel_BA_2: 0.7155 - val_loss: 0.2800 - val_avg_multilabel_BA_2: 0.7155
Epoch 93/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2722 - avg_multilabel_BA_2: 0.7155 - val_loss: 0.2797 - val_avg_multilabel_BA_2: 0.7155
Epoch 94/100
40/40 [==============================] - 0s 1ms/step - loss: 0.2722 - avg_multilabel_BA_2: 0.7155 - val_loss: 0.2794 - val_avg_multilabel_BA_2: 0.7155
Epoch 95/100
40/40 [=========